In [ ]:
np.sum(np.diff(x) * (y[:-1] + np.diff(y)/2))

In [1]:
import numpy as np

In [2]:
class Preferences(object):
    def __init__(self):
        self.end = 0
        self.endvalue = 0
        self.id = 0
        self.start = 0
        self.startvalue = 0

In [3]:
#row is agent, column is slice
preferences = [[Preferences() for i in range(3)] for j in range(3)]

In [4]:
preferences[0][0].end = 1
preferences[0][0].endvalue = 3.7
preferences[0][0].id = 1
preferences[0][0].start = 0
preferences[0][0].startvalue = 5.8

In [5]:
preferences[0][1].end = 2
preferences[0][1].endvalue = 5.5
preferences[0][1].id = 2
preferences[0][1].start = 1
preferences[0][1].startvalue = 5.5

In [6]:
preferences[0][2].end = 3
preferences[0][2].endvalue = 7
preferences[0][2].id = 3
preferences[0][2].start = 2
preferences[0][2].startvalue = 1.6

In [7]:
preferences[1][0].end = 1
preferences[1][0].endvalue = 7.3
preferences[1][0].id = 4
preferences[1][0].start = 0
preferences[1][0].startvalue = 3

In [8]:
preferences[1][1].end = 2
preferences[1][1].endvalue = 7.5
preferences[1][1].id = 5
preferences[1][1].start = 1
preferences[1][1].startvalue = 3

In [9]:
preferences[1][2].end = 3
preferences[1][2].endvalue = 0
preferences[1][2].id = 6
preferences[1][2].start = 2
preferences[1][2].startvalue = 10

In [10]:
preferences[2][0].end = 1
preferences[2][0].endvalue = 4.3
preferences[2][0].id = 7
preferences[2][0].start = 0
preferences[2][0].startvalue = 9.4

In [11]:
preferences[2][1].end = 3
preferences[2][1].endvalue = 7.6
preferences[2][1].id = 8
preferences[2][1].start = 1
preferences[2][1].startvalue = 7.6

In [12]:
preferences[2][2] = None

In [13]:
cakeSize = 3

In [14]:
#prefs, normalizationConstant = oneLipschitz(preferences, cakeSize)
preferences = oneLipschitz(preferences, cakeSize)

NameError: name 'oneLipschitz' is not defined

1.0000000000000002

In [45]:
epsilon = 0.0025

Preprocessing

Hungriness:

In [ ]:
def branzeiNisan(prefs, cakeSize, epsilon):
    preferences = oneLipschitz(prefs, cakeSize)
    alphaLowerBound = computeEquipartitionFourAgents(prefs, epsilon)
    if checkEnvyFree(prefs, alphaLowerBound) == True:
        return division(prefs, alphaLowerBound)
    alphaUpperBound = 1
    while abs(alphaUpperBound - alphaLowerBound) > ((epsilon/10)**4)/12:
        alpha = (alphaLowerBound + alphaUpperBound) / 2
        if checkConditions(prefs, alpha) == True:
            alphaLowerBound = alpha
        else:
            alphaUpperBound = alpha
    return division(prefs, alphaLowerBound)
        

In [153]:
cutBounds = find_left_cut_bounds(preferences, epsilon)

In [154]:
left = cutBounds.midpoint()
value_query(0,preferences, 0, left, epsilon) / value_query(0,preferences, left, 1, epsilon)

0.5000920435826642

In [149]:
cutBounds = find_right_cut_bounds(preferences, epsilon)

In [152]:
right = cutBounds.midpoint()
value_query(0,preferences, right, 1, epsilon) / value_query(0,preferences, 0, right, epsilon)

0.5001954804512858

In [39]:
check = Bounds(0.6344324880074275, 0.6344324880074286)

In [38]:
check.midpoint()

0.634432488007428

Ignore Below.

In [105]:
def compute_equipartition_three_agents(prefs, epsilon):
    left_cut_bounds = find_left_cut_bounds(prefs, epsilon)
    right_cut_bounds = find_right_cut_bounds(prefs, epsilon)
    equipartition = \
        exact_equipartition_cuts_three_agents(prefs, left_cut_bounds, 
                                              right_cut_bounds)

In [106]:
def find_left_cut_bounds(prefs, epsilon):
    left_cut_bounds = Bounds(0, 1)
    while abs(left_cut_bounds.upper - left_cut_bounds.lower) - epsilon > 1e-10:
        left_cut_bounds = left_cut_bounds_update(prefs, left_cut_bounds, epsilon)
    return left_cut_bounds
            

In [140]:
def left_cut_bounds_update(prefs, left_cut_bounds, epsilon):
    left_cut = left_cut_bounds.midpoint()
    segment_one_value = value_query(0, prefs, 0, left_cut, epsilon)
    right_cut = cut_query(0, prefs, left_cut, segment_one_value, 
                        epsilon, end_cut = True)
    left_cut_bounds = left_adjustment_three_agent(prefs, left_cut_bounds, 
                                                  right_cut, segment_one_value, 
                                                  epsilon)
    return left_cut_bounds

In [147]:
def find_right_cut_bounds(prefs, epsilon):
    right_cut_bounds = Bounds(0, 1)
    while abs(right_cut_bounds.upper - right_cut_bounds.lower) - epsilon > 1e-10:
        right_cut_bounds = right_cut_bounds_update(prefs, right_cut_bounds, 
                                                   epsilon)
    return right_cut_bounds

In [148]:
def right_cut_bounds_update(prefs, right_cut_bounds, epsilon):
    right_cut = right_cut_bounds.midpoint()
    segment_three_value = value_query(0, prefs, right_cut, 1, epsilon)
    left_cut = cut_query(0, prefs, right_cut, segment_three_value, 
                        epsilon, end_cut = False)
    right_cut_bounds = right_adjustment_three_agent(prefs, right_cut_bounds, 
                                                    left_cut, segment_three_value, 
                                                    epsilon)
    return right_cut_bounds

In [110]:
def exact_equipartition_cuts_three_agents(prefs, left_cut_bounds, 
                                          right_cut_bounds):
    left_cut = equipartition_left_cut_three_agents(prefs, left_cut_bounds)
    right_cut= equipartition_right_cut_three_agents(prefs, right_cut_bounds)
    equipartition = ThreeAgentPortion(left_cut, right_cut)
    return equipartition
    

In [111]:
#Must add functionality that returns if there is no cut.
def cut_query(agent, prefs, initial_cut, value, epsilon, end_cut = True):
    if end_cut == True:
        queried_cut = end_cut_query(agent, prefs, initial_cut, value, epsilon)
    else:
        queried_cut = start_cut_query(agent, prefs, initial_cut, value, epsilon)
    return queried_cut

In [112]:
def end_cut_query(agent, prefs, start, value, epsilon):
    end_cut_bounds = Bounds(start, 1)
    while abs(end_cut_bounds.upper - end_cut_bounds.lower) > 1e-15:
        end_cut_bounds = end_cut_bounds_update(prefs, start, end_cut_bounds, 
                                               value, epsilon)
    end_cut = end_cut_bounds.midpoint()
    return end_cut

In [113]:
def end_cut_bounds_update(prefs, start, end_cut_bounds, 
                          value, epsilon):
    end_cut = end_cut_bounds.midpoint()
    queried_value = valueQuery(0, prefs, start, end_cut, epsilon)
    if queried_value <= value:
        end_cut_bounds.lower  = end_cut
    if queried_value > value:
        end_cut_bounds.upper = end_cut
    return end_cut_bounds

In [114]:
def start_cut_query(agent, prefs, end, value, epsilon):
    start_cut_bounds = Bounds(0, end)
    while abs(start_cut_bounds.upper - start_cut_bounds.lower) > 1e-15:
        start_cut_bounds = start_cut_bounds_update(prefs, end, start_cut_bounds, 
                                                   value, epsilon)
    start_cut = start_cut_bounds.midpoint()
    return start_cut

In [115]:
def start_cut_bounds_update(prefs, end, start_cut_bounds, 
                            value, epsilon):
    start_cut = start_cut_bounds.midpoint()
    queried_value = valueQuery(0, prefs, start_cut, end, epsilon)
    if queried_value <= value:
        start_cut_bounds.upper  = start_cut
    if queried_value > value:
        start_cut_bounds.lower = start_cut
    return start_cut_bounds

In [116]:
def left_adjustment_three_agent(prefs, left_cut_bounds, right_cut,
                             segment_one_value, epsilon):
    left_cut = left_cut_bounds.midpoint()
    segment_three_value = valueQuery(0, prefs, right_cut, 1, epsilon)
    if segment_three_value >= segment_one_value:
        left_cut_bounds.lower = left_cut
    if segment_three_value < segment_one_value:
        left_cut_bounds.upper = left_cut
    return left_cut_bounds

In [117]:
def right_adjustment_three_agent(prefs, right_cut_bounds, left_cut, 
                                 segment_three_value, epsilon):
    right_cut = right_cut_bounds.midpoint()
    segment_one_value = valueQuery(0, prefs, 0, left_cut, epsilon)
    if segment_one_value >= segment_three_value:
        right_cut_bounds.upper = right_cut
    if segment_one_value < segment_three_value:
        right_cut_bounds.lower = right_cut
    return right_cut_bounds

In [118]:
#Remember to scale by L at end.
def one_lipschitz(prefs, cakeSize):
    prefs = change_bounds(prefs, cakeSize)
    #prefs, normalizationConstants = normalization(prefs)
    prefs = normalization(prefs)
    return prefs#, normalizationConstants

In [119]:
def change_bounds(prefs, cakeSize):
    agents = np.shape(prefs)[0]
    segments = np.shape(prefs)[1]
    for i in range(agents):
        for j in range(segments):
            if prefs[i][j] is not None:
                prefs[i][j].start /= cakeSize
                prefs[i][j].end /= cakeSize
    return prefs
        

In [120]:
'''
Ensures that each agents values the
'''
def normalization(prefs):
    agents = np.shape(prefs)[0]
    segments = np.shape(prefs)[1]
    normalization_constants = np.zeros(agents)
    for i in range(agents):
        normalization_constants[i] = value_query_initial(i, prefs, 0, 1)
        for j in range(segments):
            if prefs[i][j] is not None:
                prefs[i][j].startvalue /= normalization_constants[i]
                prefs[i][j].endvalue /= normalization_constants[i]
    return prefs#, normalizationConstants
        

In [121]:
'''
Linear interpolation on epsilon grid.
'''
def value_query(agent, prefs, start, end, epsilon):
    start_bounds, end_bounds = piecewise_linear_bounds(start, end, epsilon)
    
    if start_bounds.upper - start >= end - end_bounds.lower:
        return value_query_variant_one(agent, prefs, start, end, epsilon)
    if start_bounds.upper - start < end - end_bounds.lower:
        return value_query_variant_two(agent, prefs, start, end, epsilon)

In [122]:
def value_query_variant_one(agent, prefs, start, end, epsilon):
    start_bounds, end_bounds = piecewiseLinearBounds(start, end, epsilon)
    queries = intermediateQueriesVariantOne(agent, prefs, start_bounds, 
                                            end_bounds, epsilon)
    component_one = (((start_bounds.upper - start) - (end - end_bounds.lower)) / 
                    epsilon) * queries[0]
    component_two = ((end - end_bounds.lower) / epsilon) * queries[1]
    component_three = ((start - start_bounds.lower) / epsilon) * queries[2]
    return component_one + component_two + component_three

In [123]:
def value_query_variant_two(agent, prefs, start, end, epsilon):
    start_bounds, end_bounds = piecewise_linear_bounds(start, end, epsilon)
    queries = intermediate_queries_variant_two(agent, prefs, start_bounds, 
                                               end_bounds, epsilon)
    component_one = (((end - end_bounds.lower) - (start_bounds.upper - start)) / 
                    epsilon) * queries[0]
    component_two = ((start_bounds.upper - start) / epsilon) * queries[1]
    component_three = ((end_bounds.upper - end) / epsilon) * queries[2]
    return component_one + component_two + component_three

In [124]:
def piecewise_linear_bounds(start, end, epsilon):
    if start != 1:
        start_lower_bound = (start // epsilon) * epsilon
        start_upper_bound = start_lower_bound + epsilon 
    else:
        start_lower_bound = 1 - epsilon
        start_upper_bound = 1
    if end != 1:
        end_lower_bound = (end // epsilon) * epsilon
        end_upper_bound = end_lower_bound + epsilon
    else:
        end_lower_bound = 1 - epsilon
        end_upper_bound = 1

    start_bounds = Bounds(start_lower_bound, start_upper_bound)
    end_bounds = Bounds(end_lower_bound, end_upper_bound)

    return start_bounds, end_bounds

In [125]:
def intermediate_queries_variant_one(agent, prefs, start_bounds, 
                                     end_bounds, epsilon):
    query_one = valueQueryHungry(agent, prefs, start_bounds.lower, 
                                end_bounds.lower, epsilon)
    query_two = valueQueryHungry(agent, prefs, start_bounds.lower, 
                                end_bounds.upper, epsilon)
    query_three = valueQueryHungry(agent, prefs, start_bounds.upper, 
                                  end_bounds.lower, epsilon)
    return query_one, query_two, query_three

In [126]:
def intermediate_queries_variant_two(agent, prefs, start_bounds, 
                                     end_bounds, epsilon):
    query_one = value_query_hungry(agent, prefs, start_bounds.upper, 
                                   end_bounds.upper, epsilon)
    query_two = value_query_hungry(agent, prefs, start_bounds.lower, 
                                   end_bounds.upper, epsilon)
    query_three = value_query_hungry(agent, prefs, start_bounds.upper, 
                                     end_bounds.lower, epsilon)
    return query_one, query_two, query_three

In [127]:
def value_query_hungry(agent, prefs, start, end, epsilon):
    initial_value = value_query_initial(agent, prefs, start, end)
    value = initial_value/2 + epsilon * (end - start)
    return value

In [128]:
def value_query_initial(agent, prefs, start, end):
    check_valid_bounds(start,end)
    if end <= start:
        value = 0
    else:
        value = one_sided_query(agent, prefs, end) - \
                one_sided_query(agent, prefs, start)
    return value
    

In [129]:
def check_valid_bounds(start,end):
    assert (start >= 0 and start <= 1 and end >= 0 and end <= 1), \
        "invalid bounds. start and end should be between 0 and 1."

In [130]:
def one_sided_query(agent, prefs, end):
    segments = np.size(prefs[agent])
    value = 0
    for i in range(segments):
        if end > prefs[agent][i].end:
            value += find_full_value(prefs[agent][i])
        if end <= prefs[agent][i].end:
            value += find_partial_value(prefs[agent][i], end)
            return value

In [131]:
def find_full_value(segment):
    segment_width = segment.end - segment.start
    if segment.startvalue == segment.endvalue:
        value = segment.startvalue * segment_width
        return value
    if segment.startvalue != segment.endvalue:
        value = 0.5 * (segment.startvalue + segment.endvalue) * segment_width
        return value

In [132]:
def find_partial_value(segment, partial_segment_end):
    segment_width = partial_segment_end - segment.start
    partial_segment_end_value = interpolate(segment, partial_segment_end)
    if segment.startvalue == segment.endvalue:
        value = segment.startvalue * segment_width
        return value
    if segment.startvalue != segment.endvalue:
        value = 0.5 * (segment.startvalue + partial_segment_end_value) * \
                segment_width
        return value

In [133]:
'''
Uses the linear interpolation formula to find the associated y-coordinate for a 
given x-coordinate on a line between two points with known x and y coordinates.
'''
def interpolate(segment, x_coordinate):
    if segment.startvalue == segment.endvalue:
        y_coordinate = segment.startvalue
        return y_coordinate
    if segment.startvalue != segment.endvalue:
        y_coordinate = ((segment.startvalue * (segment.end - x_coordinate) +
                         segment.endvalue * (x_coordinate - segment.start)) / 
                        (segment.end - segment.start))                
        return y_coordinate

In [134]:
class Bounds:
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper

    def midpoint(self):
        return (self.lower + self.upper) / 2


In [137]:
class ThreeAgentPortion:
    def __init__(self, left_cut, right_cut):
        self.left = left_cut
        self.right = right_cut